# SCADA Flow Rate Calculation Model

In [63]:
import pandas as pd
import os
import csv

import re
import datetime
import io
from zipfile import ZipFile

import pandas as pd
import numpy as np
import pandas_gbq
import janitor

# Section 1

## Read SCADA data into Pandas Dataframe

In [64]:
df = pd.read_csv('Kelly Island Data Edited.csv')

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3875305116.py:1: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Kelly Island Data Edited.csv')


## Convert the DateAndTime column to Datetime Format

In [65]:
df['DateAndTime'] = pd.to_datetime(df['DateAndTime'], format="%m/%d/%Y %H:%M:%S.%f", errors = 'coerce')


## Format the dataframe

In [66]:
df.drop('Status', axis=1, inplace=True)
df.drop('Marker', axis=1, inplace=True)
df.drop('Unnamed: 6', axis=1, inplace=True)
df.drop('Unnamed: 7', axis=1, inplace=True)

In [67]:
df = df.dropna()

In [68]:
#df.head(20)

## Remove Extraneous 1's and 0's

In [69]:
# first remove 0's with the same timestamp right next to each other

for i in reversed(range(1, len(df))):
    if df.loc[i, 'Val'] == 0 and df.loc[i-1, 'Val'] == 0 and df.loc[i, 'DateAndTime'] == df.loc[i-1, 'DateAndTime']:
        # drop both rows if conditions are met
        df = df.drop([i, i-1]).reset_index(drop=True)
    

# Section 2

In this section I will create a new dataframe called calc_df, which will hold all of our data going forward. In section 2 I'll begin by filling the new dataframe with only time stamps containing pumps turning on or off (Val column 1 for on and 0 for off). From there I will convert both on and off timestamps into Unix time and calculate the difference in time between the pump turning on and off. Remove the # in front of any of the calc_df lines to see a preview of the dataframe.

## Let's set up our new dataframe

In [70]:
calc_df = pd.DataFrame()
calc_df['StartDateAndTime'] = ()
calc_df['EndDateAndTime'] = ()
calc_df['StartUnix'] = float()
calc_df['EndUnix'] = float()
calc_df['Time_diff'] = ()
calc_df['Pump_num'] = ()

## Here let's fill calc_df with On and Off times + Pump Numbers

In [71]:
has_date = False
last_index = -1
on = 1
off = 0

for index, row in df.iterrows():
    if row['Val'] == 1:
        tag = row['TagIndex']
        calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
        last_index += 1
        calc_df.at[last_index, 'Pump_num'] = tag
        #calc_df.at[last_index, 'On/Off'] = on
        has_date = True
        
    if row['Val'] == 0 and has_date:
        tag = row['TagIndex']
        calc_df.at[last_index, 'EndDateAndTime'] = row['DateAndTime']
        calc_df.at[last_index, 'Pump_num'] = tag
        #calc_df.at[last_index, 'On/Off'] = off

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning

...

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  calc_df = calc_df.append({'StartDateAndTime': row['DateAndTime']}, ignore_index=True)
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_4300/3071239915.py:9: FutureWarning

In [72]:
calc_df['EndDateAndTime'] = pd.to_datetime(calc_df['EndDateAndTime'])

#calc_df.head(20)

## Let's delete any rows that don't have either a start or end time

In [73]:
calc_df.dropna(subset=['StartDateAndTime', 'EndDateAndTime'], inplace=True)

## Now let's get Unix time for each timestamp

In [74]:
calc_df['StartUnix'] = pd.to_datetime(calc_df['StartDateAndTime']).astype(int)/ 10**9
calc_df['EndUnix'] = pd.to_datetime(calc_df['EndDateAndTime']).astype(int) / 10**9

In [75]:
#calc_df.head(20)

## Calculating the time difference between Pumps turning on and off

In [76]:
for index,row in calc_df.iterrows():
    result = (row['EndUnix']- row['StartUnix'])/60
    calc_df.at[index, 'Time_diff'] = result

In [77]:
#calc_df

# Section 3

In section 3 I will add elevation difference, volume per foot, observed pumping flow rate, average flow rate per pump, and average observed pumping flow rate (gpm).

Elevation Diference - This value can be changed for the desired lift station. Please note that each lift station will have a different elevation level. To change the elevation for a different lift station, simply change the 2.2 in the first cell below.

Observed pumping flow rate - To calculate this value we first need to input the wetwell's diameter, again this value will differ per each lift station. Simply change the diameter in the diam_well = (5) statement to the desired value. From there the vol_per_foot calculation will update and be added to the calc_df. Now we have all the information in the calc_df to calculate observed pumping flow rate. I'll take the elevation difference value times the volume per foot value, and divide the result by our time difference. The resulting value will be added to the calc_df in the observed pumping flow rate column.

Average flow rate per pump - Average flow rate per pump is calculated by adding the values in the observed pumping flow rate column for each individual pump, and then dividing that sum by the count of values for that pump. So if I iterate for 4 rows of the calc_df, and pump 1 has 3 out of the 4 rows with an observed pumping flow rate of 100, 120, and 100, the formula will sum these three rows (320) and divide by the amount of rows that were related to pump 1 (3). The average flow rate per pump 1 in row 4 would be 106.

Average observed pumping flow rate (gpm) - to calculate this value I simply add all the average flow rates for both pumps and divide the sum by the count of rows I have iterated over. For example if I have iterated over 4 rows, and 2 of the rows are pump 1 and 2 are pump 2, i'll add up the values from both pumps (100,110,100,120) and divide the sum by 4 to calculate the average observed pumping flow rate in GPM of 107.5.


## Add in Elevation Difference

In [78]:
calc_df['Elevation_dif'] = 2.2

In [79]:
#calc_df

## Calculate Observed Pumping Flow Rate

### First Lets get the volume per foot for our well

In [80]:
#Plug in diameter of wet well in ft.

diam_well = 5


#Calculate volume per foot

vol_per_foot = ((diam_well*diam_well) * 3.14)/4*7.48
vol_per_foot = round(vol_per_foot)

In [81]:
calc_df['volume_per_foot'] = vol_per_foot

In [82]:
#calc_df

## Now let's calculate our observed pumping Flow Rate

In [83]:
calc_df['observed_pumping_flow_rate'] = int()

In [84]:
for index, row in calc_df.iterrows():
    result = round((row['Elevation_dif'] * row['volume_per_foot'])/row['Time_diff'])
    calc_df.at[index,'observed_pumping_flow_rate'] = result

In [85]:
#calc_df

## Now Let's Calculate Average Flow Rate Per Pump

In [86]:
calc_df['avg_flow_rate_pump'] = int()

In [87]:
cum_sum_1 =0
cum_sum_2 =0
count_1 =0
count_2 =0

for index, row in calc_df.iterrows():
    if row['Pump_num'] == 975:
        cum_sum_1 += row['observed_pumping_flow_rate']
        count_1 += 1
        calc_df.at[index, 'avg_flow_rate_pump'] = cum_sum_1 / count_1
    if row['Pump_num'] == 974:
        cum_sum_2 += row['observed_pumping_flow_rate']
        count_2 += 1
        calc_df.at[index, 'avg_flow_rate_pump'] = cum_sum_2 / count_2
    

In [88]:
#calc_df

## Finally Let's calculate our Average Observed Pumping Flow Rate (gpm)

In [89]:
calc_df['average_observed_pumping_flow_rate'] = int()

In [90]:
cum_sum = 0
count = 0

for index, row in calc_df.iterrows():
    cum_sum += row['avg_flow_rate_pump']
    count += 1
    calc_df.at[index, 'average_observed_pumping_flow_rate'] = cum_sum / count


In [91]:
#calc_df.head(20)

# Section 4

In this section I will calculate the time to fill, inflow, and average inflow. These values need to be accounted for when calculating flow rate, as even when the pumps are turned on within a wet well and the level is dropping, water is still flowing through the pipes into the well.

Time to fill - To calculate time to fill for each row in calc_df I take the stop unix time of the first row within calc_df (when the pump turned off) and subtract this value from the start unix time off the next row (when the pump turned back on). The difference between the pump turning off and the next pump turning on is my time to fill measure.

Inflow - To calculate my inflow value for each row in calc_df, I take the row's volume per foot value and multiply by the row's elevation difference value. I then divide that result by the row's time to fill value we just calculated.

Average Inflow - To calculate my average inflow for each row in calc_df, I simply add all the values for inflow in the row's I have iterated over and divide by the count of those rows. So if I have iterated over 4 rows so far in calc_df and the inflow values are (60,40,20,60) I sum these values and divide by four to get my average inflow value in row 4, 45.

## Calculate time to fill

In [92]:
calc_df['Time_to_Fill'] = float()

In [93]:
stop_list= []

for index, row in calc_df.iterrows():
    stop = row['EndUnix']
    stop_list.append(stop)

start_list = []

for index, row in calc_df.iloc[1:].iterrows():
    start = row['StartUnix']
    start_list.append(start)
    

In [94]:
index = 1

for start, end in zip(start_list, stop_list):
    result = (start - end) /60
    #print(result)
    calc_df.at[index, 'Time_to_Fill'] = result
    index += 1

In [95]:
#calc_df

## Calculate Inflow and Average Inflow

In [96]:
calc_df['Inflow'] = float()
calc_df['Avg_Inflow'] = float()

In [97]:
for index, row in calc_df.iterrows():
    if row['Time_to_Fill'] != 0:
        inflow = row['volume_per_foot'] * row['Elevation_dif'] / row['Time_to_Fill']
        calc_df.at[index, 'Inflow'] = inflow
    else:
        pass

In [98]:
cum_sum = 0
count = 0

for index, row in calc_df.iterrows():
    cum_sum += row['Inflow']
    count += 1
    calc_df.at[index, 'Avg_Inflow'] = cum_sum / count


In [99]:
#calc_df

# Section 5

In this section I will calculate the actual pump flow rate in GPM, average flow rate per pump in GPM, and average pump flow rate in GPM.

Actual pump flow rate - To calculate actual pump flow rate for each row in calc_df, I simply take the row's observed pumping flow rate value we calculated earlier, and add the row's inflow value.

Average flow rate per pump - To calculate the average flow rate per pump I add the values in the actual pump flow rate column for each individual pump, and then divide that sum by the count of values for that pump. So if I iterate for 4 rows of the calc_df, and pump 1 has 3 out of the 4 rows with an actual pump flow rate of 100, 120, and 100, the formula will sum these three rows (320) and divide by the amount of rows that were related to pump 1 (3). The average flow rate per pump 1 in row 4 would be 106.

Average pump flow rate - to calculate this value I simply add all the actual pump flow rates for both pumps and divide the sum by the count of rows I have iterated over. For example if I have iterated over 4 rows, and 2 of the rows are pump 1 and 2 are pump 2, i'll add up the values from both pumps (100,110,100,120) and divide the sum by 4 to calculate the average pump flow rate in GPM of 107.5.

## Calculate Actual Pump Flow Rate (GPM)

In [100]:
calc_df['Actual_Pump_Flow_Rate'] = int()

In [101]:
for index, row in calc_df.iterrows():
    result = round(row['observed_pumping_flow_rate'] + row['Inflow'])
    calc_df.at[index, 'Actual_Pump_Flow_Rate'] = result

ValueError: cannot convert float NaN to integer

In [ ]:
#calc_df.head(10)

## Finally Let's calculate our Average Flow Rate Per Pump and the Average Pump Flow Rate in (GPM)

In [ ]:
calc_df['Average_Pump_1_Flow_Rate'] = int()
calc_df['Average_Pump_2_Flow_Rate'] = int()
calc_df['Average_Pump_Flow_Rate(1&2)'] = int()

In [ ]:
cum_sum = 0
count = 0

for index, row in calc_df.iterrows():
    if row['Pump_num'] == 974:
        cum_sum += row['Actual_Pump_Flow_Rate']
        count += 1
        calc_df.at[index,'Average_Pump_1_Flow_Rate'] = cum_sum / count

In [ ]:
cum_sum = 0
count = 0

for index, row in calc_df.iterrows():
    if row['Pump_num'] == 975:
        cum_sum += row['Actual_Pump_Flow_Rate']
        count += 1
        calc_df.at[index,'Average_Pump_2_Flow_Rate'] = cum_sum / count

In [ ]:
cum_sum = 0
count = 0

for index, row in calc_df.iterrows():
    cum_sum += row['Actual_Pump_Flow_Rate']
    count += 1
    calc_df.at[index,'Average_Pump_Flow_Rate(1&2)'] = cum_sum / count

In [ ]:
calc_df

In [ ]:
calc_df.to_csv("Model Data for Kelly Lift Station.csv")